In [1]:
import pandas as pd
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [4]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import gc

In [7]:
df = pd.read_csv('E:/Datasets/validation-annotations-human-imagelabels.csv', usecols=[0,2,3])

In [8]:
df = df[df.Confidence == 1]

In [9]:
import numpy  as np

In [10]:
classes = np.array(['/m/01g317', '/m/09j2d', '/m/04yx4', '/m/0dzct', '/m/07j7r', '/m/05s2s', '/m/03bt1vf', '/m/07yv9', '/m/0cgh4', '/m/01prls', '/m/09j5n', '/m/0jbk', '/m/0k4j', '/m/05r655', '/m/02wbm', '/m/0c9ph5', '/m/083wq', '/m/0c_jw', '/m/03jm5', '/m/0d4v4'])

In [11]:
li = []
for i in classes:
    li.append(df[df.LabelName == i])

In [12]:
df = pd.concat(li).sample(frac=1).reset_index(drop=True)

In [13]:
del li
gc.collect()
df.head()

,ImageID,LabelName,Confidence
0,00575b9132bb3746,/m/01g317,1
1,7c491208fa65f51c,/m/01g317,1
2,53e5eeeff0fd0bc0,/m/01g317,1
3,eb0f36c7ce3366cc,/m/01g317,1
4,218167595eb76b7b,/m/0k4j,1


In [14]:
labels = df.LabelName.tolist()
Imageid = df.ImageID.values
print(len(df))

68867


In [15]:
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Conv2D, PReLU, BatchNormalization, MaxPooling2D, Dropout, Flatten
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model, Sequential
from tqdm import tqdm_notebook
from tqdm import tqdm
from keras.optimizers import Adam

In [12]:
import sys 
from PIL import Image 

In [16]:
X_train = [np.array(load_img(r'E:\Datasets\validation\{}.jpg'.format(i),target_size=(139,139), grayscale=False))/255 for i in tqdm(Imageid[10000:20000])]

  1%|          | 55/10000 [00:06<14:45, 11.23it/s]

KeyboardInterrupt: 

  1%|          | 55/10000 [00:20<14:45, 11.23it/s]

In [69]:
X_Val = [np.array(load_img(r'E:\Datasets\validation\{}.jpg'.format(i),target_size=(139,139), grayscale=False))/255 for i in tqdm(Imageid[:2000])]

100%|██████████| 2000/2000 [02:41<00:00, 13.28it/s]


In [70]:
#classes = classes.tolist()
Y_train, Y_Val = [], []
for i in tqdm(labels[10000:20000]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_train.append(temp)
    del temp
for i in tqdm(labels[:2000]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_Val.append(temp)
    del temp
Y_train = np.array(Y_train)
Y_Val = np.array(Y_Val)
gc.collect(), Y_train[0]

100%|██████████| 2000/2000 [00:00<00:00, 21100.55it/s]


(42050,
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]))

In [71]:
myModel = InceptionV3(weights='imagenet', include_top=False,input_shape = (139,139,3))
myModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 139, 139, 3)  0                                            
__________________________________________________________________________________________________
conv2d_377 (Conv2D)             (None, 69, 69, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_377 (BatchN (None, 69, 69, 32)   96          conv2d_377[0][0]                 
__________________________________________________________________________________________________
activation_377 (Activation)     (None, 69, 69, 32)   0           batch_normalization_377[0][0]    
__________________________________________________________________________________________________
conv2d_378

In [75]:
x = myModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(100, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)

In [76]:
model = Model(inputs=myModel.input, outputs=predictions)

In [77]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 139, 139, 3)  0                                            
__________________________________________________________________________________________________
conv2d_377 (Conv2D)             (None, 69, 69, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_377 (BatchN (None, 69, 69, 32)   96          conv2d_377[0][0]                 
__________________________________________________________________________________________________
activation_377 (Activation)     (None, 69, 69, 32)   0           batch_normalization_377[0][0]    
__________________________________________________________________________________________________
conv2d_378

In [ ]:
X_train = np.array(X_train).reshape((10000,139,139,3))
X_Val = np.array(X_Val).reshape((2000,139,139,3))

In [ ]:

from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.001),
              metrics=['accuracy'])

In [67]:
plotinfo=model.fit(X_train, Y_train, validation_data=(X_Val,Y_Val), batch_size=50, epochs=10, verbose=1)

Train on 100 samples, validate on 20 samples
Epoch 1/10
100/100 [==============================] - ETA: 25s - loss: 3.3304 - acc: 0.02 - 41s 408ms/step - loss: 3.2424 - acc: 0.0600 - val_loss: 9.5518 - val_acc: 0.0500
Epoch 2/10
100/100 [==============================] - ETA: 12s - loss: 2.6382 - acc: 0.20 - 26s 259ms/step - loss: 2.8348 - acc: 0.1900 - val_loss: 2.8554 - val_acc: 0.1500
Epoch 3/10
100/100 [==============================] - ETA: 12s - loss: 2.9168 - acc: 0.28 - 26s 263ms/step - loss: 2.7229 - acc: 0.2300 - val_loss: 2.8206 - val_acc: 0.0500
Epoch 4/10
100/100 [==============================] - ETA: 12s - loss: 2.4580 - acc: 0.32 - 27s 269ms/step - loss: 2.6929 - acc: 0.2400 - val_loss: 2.7502 - val_acc: 0.0500
Epoch 5/10
100/100 [==============================] - ETA: 12s - loss: 2.6298 - acc: 0.28 - 28s 282ms/step - loss: 2.4523 - acc: 0.2900 - val_loss: 2.7955 - val_acc: 0.0500
Epoch 6/10
100/100 [==============================] - ETA: 14s - loss: 2.9278 - acc: 0.24 